### Code for the blog **[25 SQL tips to level up your data engineering skills](https://www.startdataengineering.com/post/n-sql-tips-de/)**

# [SETUP] 

In [1]:
! python ./setup.py

Cleaning up (if any existing) tpch db file tpch.db
The file tpch.db does not exist.
Creating TPCH input data at tpch.db


## Connect to DuckDB

In [2]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect("tpch.db")
%sql conn --alias duckdb

In [3]:
%%sql
show tables;

Running query in 'duckdb'

name
customer
lineitem
nation
orders
part
partsupp
region
supplier


# [Data Model]

The TPC-H data represents a car parts seller’s data warehouse, where we record orders, items that make up that order (lineitem), supplier, customer, part (parts sold), region, nation, and partsupp (parts supplier). 

Note: Have a copy of the data model as you follow along; this will help in understanding the examples provided and in answering exercise questions.

![](../../tpch_erd.png)


# Pre-requisites

1. [Basics](https://github.com/josephmachado/adv_data_transformation_in_sql/blob/main/0-basics.ipynb)
2. [CTEs](https://github.com/josephmachado/adv_data_transformation_in_sql/blob/main/concepts/CTE/CTE.ipynb)
3. [Window functions](https://github.com/josephmachado/adv_data_transformation_in_sql/blob/main/concepts/windows/windows.ipynb)
4. [Nested data types](https://github.com/josephmachado/adv_data_transformation_in_sql/blob/main/concepts/nested_data_types/using_nested_data_types.ipynb)

# SQL tips

## 1. Handy functions for common data processing scenarios

### 1.1. Need to filter on WINDOW function without CTE/Subquery use QUALIFY

In [4]:
%%sql
SELECT 
    o_orderkey, 
    o_totalprice, 
    RANK() OVER (ORDER BY o_totalprice DESC) AS price_rank
FROM orders
QUALIFY price_rank <= 10;

Running query in 'duckdb'

o_orderkey,o_totalprice,price_rank
52965,466001.28,1
29158,439687.23,2
44707,431771.98,3
59106,430619.75,4
6882,422359.65,5
57376,411255.46,6
39456,409770.83,7
17571,408345.74,8
39620,406938.36,9
35460,405742.27,10


This query ranks the orders by o_totalprice in descending order and filters the top 10 using the QUALIFY clause. 
Without QUALIFY we would have to use a subquery or CTE approach to filter on price_rank

### 1.2. Need the first/last row in a partition use DISTINCT ON

In [5]:
%%sql
SELECT DISTINCT ON (o_custkey) 
    o_custkey, 
    o_orderdate, 
    o_totalprice
FROM orders
ORDER BY o_custkey, o_orderdate DESC;

Running query in 'duckdb'

o_custkey,o_orderdate,o_totalprice
1,1998-03-29,89230.03
2,1998-05-18,50178.44
4,1998-06-24,50422.56
5,1998-04-29,177360.54
7,1997-12-20,69976.43
8,1998-07-27,174071.02
10,1998-08-02,99067.93
11,1998-01-21,16873.08
13,1998-05-04,24140.16
14,1997-11-13,185397.75


**DISTINCT ON (o_custkey)** ensures that you get only one row per customer (o_custkey).
    The ORDER BY o_custkey, o_orderdate DESC clause ensures that the query returns the most recent order (o_orderdate) for each customer based on the latest order date.

This will return the most recent order details (like o_totalprice and o_orderdate) for each customer (o_custkey).

### 1.3. STRUCT data types are sorted based on their keys from left to right

In [6]:
%%sql
WITH order_struct AS (
    SELECT 
        o_orderkey,
        STRUCT_PACK(o_orderdate, o_totalprice, o_orderkey) AS order_info
    FROM orders
)
SELECT 
    MIN(order_info) AS min_order_date,
    MAX(order_info) AS max_order_date_price
FROM order_struct;

Running query in 'duckdb'

min_order_date,max_order_date_price
"{'o_orderdate': datetime.date(1992, 1, 1), 'o_totalprice': Decimal('29084.44'), 'o_orderkey': 5607}","{'o_orderdate': datetime.date(1998, 8, 2), 'o_totalprice': Decimal('233488.50'), 'o_orderkey': 55205}"


In [9]:
%%sql
WITH order_struct AS (
    SELECT 
        o_orderkey,
        STRUCT_PACK(o_totalprice, o_orderkey) AS order_info
    FROM orders
)
SELECT 
    MIN(order_info) AS min_order_date,
    MAX(order_info) AS max_order_date_price
FROM order_struct;

Running query in 'duckdb'

min_order_date,max_order_date_price
"{'o_totalprice': Decimal('874.89'), 'o_orderkey': 35271}","{'o_totalprice': Decimal('466001.28'), 'o_orderkey': 52965}"


In the above example the `order_info` struct will be sorted based on `o_orderdate`, `o_totalprice`, `o_orderkey` in that order.
If 2 `order_info` have the same `o_orderdate` then `o_totalprice` will be used to determine the order.

### 1.4. Get the first/last element with row_number() + qualify

In [8]:
%%sql
    SELECT 
        o_custkey, 
        o_orderdate, 
        o_totalprice,
        ROW_NUMBER() OVER (PARTITION BY o_custkey ORDER BY o_orderdate DESC) AS rn
    FROM orders
QUALIFY rn = 1;

Running query in 'duckdb'

o_custkey,o_orderdate,o_totalprice,rn
4,1998-06-24,50422.56,1
23,1997-12-14,67702.22,1
40,1998-04-13,233887.29,1
43,1998-06-30,189596.87,1
56,1998-07-29,118293.79,1
136,1998-07-04,27913.20,1
193,1997-11-15,285358.60,1
196,1998-03-21,253530.03,1
203,1997-07-19,85191.24,1
206,1998-07-27,133975.94,1


In the above example we are partitioning the data by `o_custkey` and ranking them in descing order of `o_orderdate`.

### 1.5. Check if atleast one or all boolean values are true with BOOL_OR & BOOL_AND respectively

In [13]:
%%sql
SELECT 
    o_custkey, 
    BOOL_OR(cast(o_shippriority as boolean)) AS has_atleast_one_priority_order,
    BOOL_AND(cast(o_shippriority as boolean)) AS has_all_priority_order
FROM orders
GROUP BY o_custkey;

Running query in 'duckdb'

o_custkey,has_atleast_one_priority_order,has_all_priority_order
1,False,False
2,False,False
4,False,False
5,False,False
7,False,False
8,False,False
10,False,False
11,False,False
13,False,False
14,False,False


Explanation:
* BOOL_OR(o_ispriority): This aggregate function checks whether **atleast one** of the orders for a customer (o_custkey) has a TRUE value in the o_ispriority column.
* BOOL_AND(o_ispriority): This aggregate function checks whether **all** of the orders for a customer (o_custkey) has a TRUE value in the o_ispriority column.


### 1.6. Want to select all columns except a few, use EXCLUDE

In [14]:
%%sql
SELECT * EXCLUDE (o_orderdate, o_totalprice)
FROM orders;

Running query in 'duckdb'

o_orderkey,o_custkey,o_orderstatus,o_orderpriority,o_clerk,o_shippriority,o_comment
1,370,O,5-LOW,Clerk#000000951,0,ly express platelets. deposits acc
2,781,O,1-URGENT,Clerk#000000880,0,ve the furiously fluffy dependencies. carefully regular
3,1234,F,5-LOW,Clerk#000000955,0,after the asymptotes. instructions cajole after the foxes. carefully unu
4,1369,O,5-LOW,Clerk#000000124,0,st the furiously bold pinto beans. furiously pending theodolites cajol
5,445,F,5-LOW,Clerk#000000925,0,onic requests. carefully daring foxes among the carefu
6,557,F,4-NOT SPECIFIED,Clerk#000000058,0,furiously ironic accounts haggle blithely carefully regular de
7,392,O,2-HIGH,Clerk#000000470,0,", ironic packages wa"
32,1301,O,2-HIGH,Clerk#000000616,0,"ly about the carefully express theodolites. ironic, iron"
33,670,F,3-MEDIUM,Clerk#000000409,0,"careful, regular courts. unusual"
34,611,O,3-MEDIUM,Clerk#000000223,0,osits according to the ideas are furiously final requests? slyly pe


SELECT * EXCLUDE (o_orderdate, o_totalprice): This syntax selects all columns from the orders table except the specified columns (o_orderdate and o_totalprice).

### 1.7. Tired of creating a long list of columns from GROUP BY, use GROUP BY ALL

In [15]:
%%sql
SELECT 
    o_orderkey, 
    o_custkey, 
    o_orderstatus, 
    SUM(o_totalprice) AS total_price
FROM orders
GROUP BY ALL;

Running query in 'duckdb'

o_orderkey,o_custkey,o_orderstatus,total_price
4,1369,O,56000.91
67,568,O,182481.16
68,286,O,301968.79
160,826,O,114742.32
161,167,F,17668.60
162,142,O,3553.15
163,878,O,177809.13
166,1079,O,158207.39
193,791,F,80834.26
227,100,O,69020.68


Explanation:

* GROUP BY ALL: This automatically groups by all non-aggregated columns in the SELECT statement (in this case, o_orderkey, o_custkey, and o_orderstatus).
* SUM(o_totalprice): This is the aggregated column, so DuckDB will group by the remaining columns without you needing to explicitly list them.

Using GROUP BY ALL prevents errors where you might forget to include one or more non-aggregated columns in the GROUP BY clause, making your SQL query more concise and less error-prone.

This feature is particularly useful in queries with many columns, where manually writing out all group-by columns can become tedious and prone to mistakes.

### 1.8. Need to count rows only if a certain condition is met? Use COUNT IF

In [16]:
%%sql
SELECT 
    o_custkey, 
    COUNT_IF(o_totalprice > 100000) AS high_value_orders,
    COUNT(o_totalprice) as all_orders
FROM orders
GROUP BY o_custkey;

Running query in 'duckdb'

o_custkey,high_value_orders,all_orders
1,5,9
2,6,10
4,21,31
5,5,9
7,18,24
8,12,14
10,17,27
11,4,7
13,11,21
14,9,11


This query groups by o_custkey and counts the number of orders for each customer where the total price is greater than 100,000.

This is a powerful function that simplifies counting conditional occurrences in SQL. Without `COUNT_IF` you'd have to use `SUM(CASE WHEN o_totalprice > 100000 THEN 1 ELSE 0 END)`.

### 1.9. Need to concatenate rows of string after GROUP BY? Use STRING_AGG

In [17]:
%%sql
SELECT STRING_AGG(c_name, ', ') AS customer_names
FROM customer;

Running query in 'duckdb'

customer_names


This query uses `STRING_AGG` in DuckDB to concatenate all the values from the `c_name` column in the `customer` table into a single string, separated by commas (`', '`), and returns it as `customer_names`.

### 1.10. Handle Null column values with other columns or fallback values using COALESCE

In [ ]:
%%sql
WITH fake_orders AS (
    SELECT 1 AS o_orderkey, 100 AS o_totalprice, NULL AS discount
    UNION ALL
    SELECT 2 AS o_orderkey, 200 AS o_totalprice, 20 AS discount
    UNION ALL
    SELECT 3 AS o_orderkey, 300 AS o_totalprice, NULL AS discount
)
SELECT 
    o_orderkey, 
    o_totalprice, 
    discount,
    COALESCE(discount, o_totalprice * 0.10) AS final_discount
FROM fake_orders;


Explanation:
* **COALESCE**: This function returns the first non-NULL value from a list of arguments.
* In this example, if o_totalprice is NULL for any row, COALESCE replaces it with 0. Otherwise, it returns o_totalprice.
* The query selects the o_orderkey and replaces NULL values in o_totalprice with 0.

Use Case for COALESCE:

* **Handling NULL Values**: When working with columns that may have NULL values, COALESCE can be used to substitute them with a default value (e.g., 0, '', or a custom string). This is particularly useful in financial calculations, reports, or data cleaning where NULL values need to be replaced with meaningful defaults to avoid errors or incorrect results.
* **Fallback Values**: It can also be used to provide fallback options in queries, returning alternative values if the primary value is NULL.

### 1.11. Generate a range of number/date rows with GENERATE_SERIES

In [18]:
%%sql
SELECT *
FROM generate_series(1, 10);

Running query in 'duckdb'

generate_series
1
2
3
4
5
6
7
8
9
10


In [19]:
%%sql
SELECT *
FROM generate_series('2024-01-01'::DATE, '2024-01-10'::DATE, INTERVAL 1 DAY);


Running query in 'duckdb'

generate_series
2024-01-01 00:00:00
2024-01-02 00:00:00
2024-01-03 00:00:00
2024-01-04 00:00:00
2024-01-05 00:00:00
2024-01-06 00:00:00
2024-01-07 00:00:00
2024-01-08 00:00:00
2024-01-09 00:00:00
2024-01-10 00:00:00


Explanation:

* **generate_series(1, 10)**: This function generates a series of numbers from 1 to 10. Each number in the range is output as a separate row
* The result will be a table with a single column of integers, from 1 to 10.

Use Case:

* **Data Simulation**: generate_series is often used to create a sequence of numbers for testing or simulating data (e.g., generating dates, IDs, or time intervals).
* **Joining with Other Tables**: You can use generate_series to produce rows that can be joined with other tables, for example, to fill in missing dates or create sequences of data where needed.
* **Looping/Iteration**: It's useful when you need to perform actions over a range of values, like generating monthly or yearly reports for a range of dates.

### 1.12. Convert ARRAY/LIST of elements to individual rows with UNNEST 

In [20]:
%%sql
WITH nested_data AS (
    SELECT 1 AS id, [10, 20, 30] AS values
    UNION ALL
    SELECT 2 AS id, [40, 50] AS values
)
SELECT 
    id, 
    UNNEST(values) AS flattened_value
FROM nested_data;


Running query in 'duckdb'

id,flattened_value
1,10
1,20
1,30
2,40
2,50


Explanation:

* **UNNEST(values)**: The UNNEST function takes the array in the values column and returns one row for each element in the array.

Use Case for UNNEST:

* **Nested Arrays**: Often used when working with semi-structured data like JSON arrays or nested lists, where you need to process each item in the array individually.

## 2. Get rows in one table depending on its presence/absence in another with SET operations

### 2.1. Get data from a table based on existence of data in another with EXISTS

In [21]:
%%sql
SELECT 
    c_custkey, 
    c_name
FROM customer 
WHERE EXISTS (
    SELECT o_orderkey
    FROM orders
    WHERE o_totalprice > 5000000 
    -- o_custkey = c_custkey  -- replace the above filter with this => Does the customer from customer table have atleast one order?
);

Running query in 'duckdb'

c_custkey,c_name


The query returns customers who have at least one order whose `o_totalprice` is greater than *5,000,000*.

**The EXISTS operator** tests for the existence of any row inside the subquery. It returns either true when the subquery returns one or more records, and false otherwise.

### 2.2. Get data that is present in both the tables with INTERSECT

In [22]:
%%sql
SELECT c_custkey 
FROM customer
INTERSECT
SELECT o_custkey 
FROM orders;

Running query in 'duckdb'

c_custkey
1
25
95
100
106
121
133
148
157
214


 The query returns customers (c_custkey) who appear in both the customer table and the orders table.

### 2.3. Get data that is present in table 1 but not in table 2 with EXCEPT

In [23]:
%%sql
SELECT c_custkey
FROM customer
EXCEPT
SELECT o_custkey
FROM orders o;

Running query in 'duckdb'

c_custkey
42
108
150
159
174
192
195
207
246
309


The query returns customers (c_custkey) that are present in the customer table but do not have corresponding records in the orders table. The EXCEPT operator removes rows that have matches in the second query.

### 2.4. Get data diff (aka delta), with (A - B) U (B - A)

In [24]:
%%sql
-- ASSUME cust_test is the customer data's next data load
DROP TABLE IF EXISTS cust_test;
-- 1. Create cust_test table from customer
CREATE TABLE cust_test AS SELECT * FROM customer;

-- 2. Append a new row to cust_test
-- Insert a new row with values for all columns
INSERT INTO cust_test VALUES (9999, 'New Customer', 'new_customer@example.com', '123', '2024-10-21', 10.00, 'ExtraColumn2', 'ExtraColumn3');

-- 3. Delete a row from cust_test (delete where customer_id = 2)
DELETE FROM cust_test WHERE c_custkey = 2;

-- 4. Update a row in cust_test (update customer with customer_id = 1)
UPDATE cust_test
SET c_name = 'Updated Name', c_address = 'updated address'
WHERE c_custkey = 1;


Running query in 'duckdb'

Count
1


In [25]:
%%sql

SELECT c_custkey, 'DELETED' as ops FROM ( 
SELECT c_custkey
FROM customer
EXCEPT
SELECT c_custkey
FROM cust_test)

UNION ALL

SELECT c_custkey, 'UPSERTED' as ops FROM ( 
SELECT c_custkey, c_name, c_address
FROM cust_test
EXCEPT
SELECT c_custkey, c_name, c_address
FROM customer)



Running query in 'duckdb'

c_custkey,ops
2,DELETED
9999,UPSERTED
1,UPSERTED


## 3. Create re-usable functions in SQL

### 3.1. Functions in SQL are called MACROs

In [ ]:
%%sql
CREATE MACRO IF NOT EXISTS percentage(numerator, denominator) AS (
    (CAST(numerator AS DOUBLE) / CAST(denominator AS DOUBLE)) * 100
);


In [ ]:
%%sql
SELECT o_orderkey, o_totalprice, percentage(o_totalprice, 50000) AS discount_percentage
FROM orders
LIMIT 5;


Explanation:
* The macro percentage takes two parameters, numerator and denominator, and returns the percentage calculation.
* In this case, it calculates the percentage of o_totalprice against a constant value (50,000).
* The macro is used like a function but expanded inline when the query is executed.

In [ ]:
%%sql
CREATE MACRO IF NOT EXISTS large_order(order_price) AS (
    CASE
        WHEN order_price > 100000 THEN 'Large Order'
        ELSE 'Regular Order'
    END
);

In [ ]:
%%sql
SELECT o_orderkey, o_totalprice, large_order(o_totalprice) AS order_type
FROM orders
LIMIT 5;

Summary:
* SQL Macros are useful for encapsulating reusable SQL expressions, similar to functions in other programming languages.
* They allow you to abstract complex logic and reuse it in different parts of your query, improving both readability and maintainability.
* DuckDB macros are inline, meaning they are expanded at query execution time, which avoids the overhead of function calls.

## 4. Dynamically generate SQL with Python

### 4.1. Use Jinja2 to create SQL queries in Python

In [26]:
from jinja2 import Template

# Define a Jinja2 SQL template with a loop
sql_template = """
SELECT o_orderkey, o_custkey, o_totalprice
FROM orders
WHERE o_totalprice > {{ price_threshold }}
{% if customer_keys %}
  AND o_custkey IN (
    {% for custkey in customer_keys %}
      {{ custkey }}{% if not loop.last %}, {% endif %}
    {% endfor %}
  )
{% endif %}
ORDER BY o_totalprice DESC;
"""

# Render the template with dynamic parameters
template = Template(sql_template)

# Parameters to be passed to the template
params = {
    "price_threshold": 20000,
    "customer_keys": [1001, 1002, 1003]  # A list of customer keys to filter on
}

# Render the SQL query (do not execute, just generate SQL)
rendered_sql = template.render(params)

# Output the generated SQL
print("Generated SQL Query:")
print("====================")
print(rendered_sql)


Generated SQL Query:

SELECT o_orderkey, o_custkey, o_totalprice
FROM orders
WHERE o_totalprice > 20000

  AND o_custkey IN (
    
      1001, 
    
      1002, 
    
      1003
    
  )

ORDER BY o_totalprice DESC;


Explanation:

* Jinja2 Loop: The {% for custkey in customer_keys %} loop dynamically generates the IN clause, listing the customer keys separated by commas.
* Conditional Logic: The {% if customer_keys %} block ensures that the IN clause is only added if the customer_keys list is not empty.
* loop.last: This Jinja2 variable is used to avoid adding a comma after the last item in the list.

## 5. Access data about your data (aka metadata)

See your db documentation to see where this data is stored. For DuckDB **[checkout their docs here](https://duckdb.org/docs/sql/meta/information_schema)**.

### 5.1. Databases store metadata in information_schema

In [27]:
%%sql
-- Information about our tables are stored here
SELECT schema_name,
    view_name
    FROM duckdb_views();

Running query in 'duckdb'

schema_name,view_name
information_schema,character_sets
information_schema,columns
information_schema,key_column_usage
information_schema,referential_constraints
information_schema,schemata
information_schema,tables
information_schema,table_constraints
main,duckdb_columns
main,duckdb_constraints
main,duckdb_databases


In [ ]:
%%sql
select * from information_schema.tables;

In [ ]:
%%sql
-- database level settings
SELECT * FROM duckdb_settings();

In [ ]:
%%sql
-- list of all tables in our DuckDB 
SELECT schema_name,
    table_name
    FROM duckdb_tables();

## 6. Avoid data duplicates with UPSERTS (aka MERGE INTO)

### 6.1. Insert new data, Update existing data in a table with UPSERT/MERGE INTO

In [28]:
%%sql
DROP TABLE IF EXISTS dim_customer_scd2;
-- Create a Slowly Changing Dimension (SCD Type 2) table for customer
CREATE TABLE dim_customer_scd2 (
    c_custkey INTEGER PRIMARY KEY,
    c_name VARCHAR,
    c_address VARCHAR,
    c_nationkey INTEGER,
    c_phone VARCHAR,
    c_acctbal DOUBLE,
    c_mktsegment VARCHAR,
    c_comment VARCHAR,
    valid_from DATE,
    valid_to DATE,
    is_current BOOLEAN
);

-- Insert current data from the TPCH customer table into the SCD2 table
INSERT INTO dim_customer_scd2
SELECT 
    c_custkey, 
    c_name, 
    c_address, 
    c_nationkey, 
    c_phone, 
    c_acctbal, 
    c_mktsegment, 
    c_comment,
    '2024-10-17' AS valid_from,
    NULL AS valid_to,  -- NULL means it's the current active record
    TRUE AS is_current
FROM customer;

Running query in 'duckdb'

Count
1500


In [29]:
%%sql
select * from dim_customer_scd2 order by c_custkey desc limit 2 ;

Running query in 'duckdb'

c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment,valid_from,valid_to,is_current
1500,Customer#000001500,XsQC6tx467elIdbQExWX,5,15-200-872-4790,6910.79,MACHINERY,pending theodolites haggle boldly after the pending accounts. caref,2024-10-17,None,True
1499,Customer#000001499,"X0UH4dmALDy3GvJqTKz,fR7O4iFQhdxWPy",3,13-273-527-9609,9128.69,AUTOMOBILE,ts are. regular asymptotes wake evenly regular accounts. rut,2024-10-17,None,True


In [30]:
%%sql
INSERT INTO dim_customer_scd2 (
    c_custkey, 
    c_name, 
    c_address, 
    c_nationkey, 
    c_phone, 
    c_acctbal, 
    c_mktsegment, 
    c_comment, 
    valid_from, 
    valid_to, 
    is_current
)
VALUES
    (1, 'Customer#000000001', 'New Address 1', 15, '25-989-741-2988', 711.56, 'BUILDING', 'comment1', '2024-10-18', NULL, TRUE),
    (2, 'Customer#000000002', 'New Address 2', 18, '12-423-790-3665', 879.49, 'FURNITURE', 'comment2', '2024-10-18', NULL, TRUE),
    (1501, 'Customer#000001501', 'New Address 1501', 24, '11-345-678-9012', 500.50, 'MACHINERY', 'comment1501', '2024-10-18', NULL, TRUE),
    (1502, 'Customer#000001502', 'New Address 1502', 21, '22-456-789-0123', 600.75, 'AUTOMOBILE', 'comment1502', '2024-10-18', NULL, TRUE)
ON CONFLICT (c_custkey) DO 
-- Handle existing customers (Customer#000000001 and Customer#000000002) for SCD Type 2
UPDATE SET valid_to = EXCLUDED.valid_from, is_current = FALSE
WHERE dim_customer_scd2.c_custkey = EXCLUDED.c_custkey AND dim_customer_scd2.is_current = TRUE;

Running query in 'duckdb'

Count
4


In [31]:
%%sql
SELECT * from dim_customer_scd2
WHERE c_custkey in (1,2,1501, 1502)
order by c_custkey;

Running query in 'duckdb'

c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment,valid_from,valid_to,is_current
1,Customer#000000001,j5JsirBM9PsCy0O1m,15,25-989-741-2988,711.56,BUILDING,y final requests wake slyly quickly special accounts. blithely,2024-10-17,2024-10-18,False
2,Customer#000000002,487LW1dovn6Q4dMVymKwwLE9OKf3QG,13,23-768-687-3665,121.65,AUTOMOBILE,y carefully regular foxes. slyly regular requests about the bli,2024-10-17,2024-10-18,False
1501,Customer#000001501,New Address 1501,24,11-345-678-9012,500.5,MACHINERY,comment1501,2024-10-18,None,True
1502,Customer#000001502,New Address 1502,21,22-456-789-0123,600.75,AUTOMOBILE,comment1502,2024-10-18,None,True


In the above example, we can see that we create an **[SCD2](https://www.startdataengineering.com/post/how-to-join-fact-scd2-tables/#what-is-an-scd2-table-and-why-use-it)** table and UPSERT new data into it using UPSERT.

**Note** Some DBs have `INSERT..ON CONFLICT` and some have access to `MERGE INTO..` Check your DB/Table format documentation for details.                                                            

## 7. Advanced JOIN types

### 7.1. Get value from table2 that is closest(in time) to table1's row with ASOF JOIN 

In [32]:
%%sql
WITH stock_prices AS (
    SELECT 'APPL' AS ticker, TIMESTAMP '2001-01-01 00:00:00' AS "when", 1 AS price
    UNION ALL
    SELECT 'APPL', TIMESTAMP '2001-01-01 00:01:00', 2
    UNION ALL
    SELECT 'APPL', TIMESTAMP '2001-01-01 00:02:00', 3
    UNION ALL
    SELECT 'MSFT', TIMESTAMP '2001-01-01 00:00:00', 1
    UNION ALL
    SELECT 'MSFT', TIMESTAMP '2001-01-01 00:01:00', 2
    UNION ALL
    SELECT 'MSFT', TIMESTAMP '2001-01-01 00:02:00', 3
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2001-01-01 00:00:00', 1
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2001-01-01 00:01:00', 2
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2001-01-01 00:02:00', 3
),
portfolio_holdings AS (
    SELECT 'APPL' AS ticker, TIMESTAMP '2000-12-31 23:59:30' AS "when", 5.16 AS shares
    UNION ALL
    SELECT 'APPL', TIMESTAMP '2001-01-01 00:00:30', 2.94
    UNION ALL
    SELECT 'APPL', TIMESTAMP '2001-01-01 00:01:30', 24.13
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2000-12-31 23:59:30', 9.33
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2001-01-01 00:00:30', 23.45
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2001-01-01 00:01:30', 10.58
    UNION ALL
    SELECT 'DATA', TIMESTAMP '2000-12-31 23:59:30', 6.65
    UNION ALL
    SELECT 'DATA', TIMESTAMP '2001-01-01 00:00:30', 17.95
    UNION ALL
    SELECT 'DATA', TIMESTAMP '2001-01-01 00:01:30', 18.37
)
SELECT h.ticker,
    h.when,
    p.when as stock_price_ts,
    price,
    shares,
    price * shares AS value
FROM portfolio_holdings h
ASOF JOIN stock_prices p
       ON h.ticker = p.ticker
      AND h.when >= p.when
ORDER BY 1, 2;

Running query in 'duckdb'

ticker,when,stock_price_ts,price,shares,value
APPL,2001-01-01 00:00:30,2001-01-01 00:00:00,1,2.94,2.94
APPL,2001-01-01 00:01:30,2001-01-01 00:01:00,2,24.13,48.26
GOOG,2001-01-01 00:00:30,2001-01-01 00:00:00,1,23.45,23.45
GOOG,2001-01-01 00:01:30,2001-01-01 00:01:00,2,10.58,21.16


AsOf joins are used to solve is finding the value of a varying property at a specific point in time. This use case is so common that it is where the name came from:

**Give me the value of the property as of this time.**

In the above example, note that even thought rows in `portfolio_holdings` match with multiple rows in `stock_prices` we only pick the row from `stock_price` that is closest(in time) to row in `portfolio_holdings`.

In [33]:
%%sql
WITH stock_prices AS (
    SELECT 'APPL' AS ticker, TIMESTAMP '2001-01-01 00:00:00' AS "when", 1 AS price
    UNION ALL
    SELECT 'APPL', TIMESTAMP '2001-01-01 00:01:00', 2
    UNION ALL
    SELECT 'APPL', TIMESTAMP '2001-01-01 00:02:00', 3
    UNION ALL
    SELECT 'MSFT', TIMESTAMP '2001-01-01 00:00:00', 1
    UNION ALL
    SELECT 'MSFT', TIMESTAMP '2001-01-01 00:01:00', 2
    UNION ALL
    SELECT 'MSFT', TIMESTAMP '2001-01-01 00:02:00', 3
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2001-01-01 00:00:00', 1
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2001-01-01 00:01:00', 2
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2001-01-01 00:02:00', 3
),
portfolio_holdings AS (
    SELECT 'APPL' AS ticker, TIMESTAMP '2000-12-31 23:59:30' AS "when", 5.16 AS shares
    UNION ALL
    SELECT 'APPL', TIMESTAMP '2001-01-01 00:00:30', 2.94
    UNION ALL
    SELECT 'APPL', TIMESTAMP '2001-01-01 00:01:30', 24.13
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2000-12-31 23:59:30', 9.33
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2001-01-01 00:00:30', 23.45
    UNION ALL
    SELECT 'GOOG', TIMESTAMP '2001-01-01 00:01:30', 10.58
    UNION ALL
    SELECT 'DATA', TIMESTAMP '2000-12-31 23:59:30', 6.65
    UNION ALL
    SELECT 'DATA', TIMESTAMP '2001-01-01 00:00:30', 17.95
    UNION ALL
    SELECT 'DATA', TIMESTAMP '2001-01-01 00:01:30', 18.37
)
SELECT h.ticker,
    h.when,
    p.when as stock_price_ts,
    price,
    shares,
    price * shares AS value
FROM portfolio_holdings h
 JOIN stock_prices p
       ON h.ticker = p.ticker
      AND h.when >= p.when
ORDER BY 1, 2;

Running query in 'duckdb'

ticker,when,stock_price_ts,price,shares,value
APPL,2001-01-01 00:00:30,2001-01-01 00:00:00,1,2.94,2.94
APPL,2001-01-01 00:01:30,2001-01-01 00:00:00,1,24.13,24.13
APPL,2001-01-01 00:01:30,2001-01-01 00:01:00,2,24.13,48.26
GOOG,2001-01-01 00:00:30,2001-01-01 00:00:00,1,23.45,23.45
GOOG,2001-01-01 00:01:30,2001-01-01 00:00:00,1,10.58,10.58
GOOG,2001-01-01 00:01:30,2001-01-01 00:01:00,2,10.58,21.16


If we do not use `asof` join, we will have to use windows to filter out data. 
![as of join](./images/asof.png)

### 7.2. Get rows in table1 that are not in table2 with ANTI JOIN

In [34]:
%%sql
SELECT c.c_custkey
FROM customer c
LEFT JOIN orders o
ON c.c_custkey = o.o_custkey
WHERE o.o_custkey IS NULL
ORDER BY c.c_custkey
LIMIT 5;

Running query in 'duckdb'

c_custkey
3
6
9
12
15


In [35]:
%%sql
-- some DBs have inbuilt support for left anti join
SELECT c.c_custkey
FROM customer c
ANTI JOIN orders o
ON c.c_custkey = o.o_custkey
ORDER BY c.c_custkey
LIMIT 5;

Running query in 'duckdb'

c_custkey
3
6
9
12
15


### 7.3. For every row in table1 join will all the "matching" rows in table2 with LATERAL JOIN

In [36]:
%%sql
SELECT 
    o.o_orderkey, 
    o.o_totalprice, 
    l.l_linenumber,
    l.l_extendedprice
FROM orders o,
LATERAL (
    SELECT l.l_linenumber,
    l_extendedprice
    FROM lineitem l
    WHERE l.l_orderkey = o.o_orderkey
    and l.l_linenumber <= 2
    and l.l_extendedprice < (o.o_totalprice/2)
) AS l
    ORDER BY 1, 3;

Running query in 'duckdb'

o_orderkey,o_totalprice,l_linenumber,l_extendedprice
1,172799.49,1,24710.35
1,172799.49,2,56688.12
3,205654.30,1,42436.80
3,205654.30,2,53468.31
5,105367.67,1,14806.20
5,105367.67,2,29672.24
7,271885.66,1,20673.84
7,271885.66,2,12190.05
32,198665.57,1,48406.96
32,198665.57,2,60223.36


For each row in the orders table (o), the subquery in the LATERAL JOIN selects line items (l) that match certain conditions.

In [37]:
%%sql
SELECT *
FROM range(3) t(i), LATERAL (SELECT i + 1) t2(j);

Running query in 'duckdb'

i,j
0,1
2,3
1,2


In [38]:
%%sql
SELECT 
    o.o_orderkey, 
    o.o_totalprice, 
    l.lineitem_count
FROM orders o,
LATERAL (
    SELECT COUNT(*) AS lineitem_count
    FROM lineitem l
    WHERE l.l_orderkey = o.o_orderkey
) AS l;

Running query in 'duckdb'

o_orderkey,o_totalprice,lineitem_count
1,172799.49,6
32,198665.57,6
33,146567.24,4
68,301968.79,7
98,71721.40,4
133,95971.06,4
164,250417.20,7
167,64017.85,2
352,25542.02,1
356,189160.02,5


For each row in the orders table (o), the subquery in the LATERAL JOIN counts the number of line items (lineitem_count) related to that order.


## 8. Business use cases

### 8.1. Change dimension values to individual columns with PIVOT

In [39]:
%%sql
SELECT 
    o_custkey,
    SUM(CASE WHEN o_orderstatus = 'F' THEN o_totalprice ELSE 0 END) AS fulfilled_total,
    SUM(CASE WHEN o_orderstatus = 'O' THEN o_totalprice ELSE 0 END) AS open_total,
    SUM(CASE WHEN o_orderstatus = 'P' THEN o_totalprice ELSE 0 END) AS pending_total
FROM orders
GROUP BY o_custkey
ORDER BY o_custkey;

Running query in 'duckdb'

o_custkey,fulfilled_total,open_total,pending_total
1,286642.08,1142231.53,0.00
2,631323.16,525181.76,0.00
4,1208443.25,2926124.14,0.00
5,435083.27,648959.47,0.00
7,1840220.65,2081800.33,0.00
8,1288436.76,1023161.99,0.00
10,2248225.63,1616872.55,0.00
11,804864.98,428621.55,0.00
13,1028321.46,1264513.11,0.00
14,968245.45,819113.63,0.00


Explanation:

* SUM(CASE WHEN o_orderstatus = 'F' THEN o_totalprice ELSE 0 END): This sums up the o_totalprice for orders that have a status of 'F' (Fulfilled) for each customer (o_custkey).
* Similarly, we apply this for other statuses like 'O' (Open) and 'P' (Pending) to pivot the data by o_orderstatus.
* GROUP BY o_custkey: Groups the data by each customer to aggregate the total prices based on order status.

In [40]:
%%sql
-- some DBs have support for PIVOT
FROM orders
PIVOT (
    sum(o_totalprice)
    FOR
        o_orderstatus IN ('F', 'O', 'P')
    GROUP BY o_custkey
)
    ORDER BY o_custkey;

Running query in 'duckdb'

o_custkey,F,O,P
1,286642.08,1142231.53,None
2,631323.16,525181.76,None
4,1208443.25,2926124.14,None
5,435083.27,648959.47,None
7,1840220.65,2081800.33,None
8,1288436.76,1023161.99,None
10,2248225.63,1616872.55,None
11,804864.98,428621.55,None
13,1028321.46,1264513.11,None
14,968245.45,819113.63,None


### 8.2. Generate metrics for every possible combination of dimensions with CUBE

In [41]:
%%sql
SELECT 
    o_orderpriority,
    o_orderstatus,
    EXTRACT(YEAR FROM o_orderdate) AS order_year,
    SUM(o_totalprice) AS total_sales
FROM orders
GROUP BY CUBE (o_orderpriority, o_orderstatus, order_year)
ORDER BY 1,2,3;

Running query in 'duckdb'

o_orderpriority,o_orderstatus,order_year,total_sales
1-URGENT,F,1992,65037070.10
1-URGENT,F,1993,60503901.99
1-URGENT,F,1994,68381440.65
1-URGENT,F,1995,12186862.02
1-URGENT,F,None,206109274.76
1-URGENT,O,1995,40259411.38
1-URGENT,O,1996,64262978.01
1-URGENT,O,1997,67931485.24
1-URGENT,O,1998,36552106.60
1-URGENT,O,None,209005981.23


Explanation:

* CUBE (o_custkey, o_orderstatus, order_year): This will group the data and calculate subtotals and grand totals for all possible combinations of o_custkey, o_orderstatus, and order_year.
* It will generate all combinations of the grouping columns, including:
    * Grouping by just o_custkey
    * Grouping by just o_orderstatus
    * Grouping by just order_year
    * Grouping by all three together
    * Grouping by pairs of columns
    * A grand total (no grouping by any column)
    * SUM(o_totalprice): For each combination of groupings, it sums the total order price.

Example Output:

The query will return something like the following:
| o_custkey | o_orderstatus | order_year | total_sales |
|-----------|---------------|------------|-------------|
| 1001      | O             | 2024       | 20000.00    |
| 1001      | F             | 2023       | 15000.00    |
| 1001      | O             | NULL       | 35000.00    |
| 1002      | NULL          | 2024       | 30000.00    |
| NULL      | O             | 2024       | 50000.00    |
| NULL      | NULL          | NULL       | 100000.00   |


Grand Total: The row with all NULL values represents the grand total (total_sales for all customers, all statuses, and all years).
                                                                      
Subtotals:
                                                                      
* The row with o_custkey = 1001, o_orderstatus = 'O', and order_year = NULL represents the subtotal of orders for customer 1001 and status 'O' across all years.
* The row with o_custkey = NULL, o_orderstatus = 'O', and order_year = 2024 represents the subtotal for all customers with status 'O' in the year 2024.

Use Case:

1. OLAP Reporting: CUBE is commonly used in OLAP scenarios where you need to analyze data from multiple perspectives. For instance, you may want to generate reports that show total sales by customer, by order status, by year, and all possible combinations of these dimensions.
2. Sales Analysis: In sales analysis, CUBE can help create pivot-like summaries that show how different attributes (e.g., region, product, time period) contribute to the overall sales.
3. Financial Reports: Financial departments often use CUBE to calculate totals and subtotals across dimensions like departments, time periods, and account categories, making it easier to prepare comprehensive financial reports.

This is a powerful tool for producing multidimensional summaries of data in one go, helping with complex reporting and data analysis tasks.